In [1]:
!pip install --upgrade pip
!pip install transformers==4.47.1 jiwer soundfile evaluate bnunicodenormalizer
!apt install git-lfs

   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 258.0 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/10.1 MB 258.0 kB/s eta 0:00:38
  

'apt' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [2]:
import wget
print(wget)


<module 'wget' from 'c:\\Users\\USER\\Lib\\site-packages\\wget.py'>


In [5]:
# Download the first dataset
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/74p9d5frg3-1.zip -O /kaggle/working/dataset.zip
!unzip /kaggle/working/dataset.zip -d /kaggle/working/YembaEGRA
!rm /kaggle/working/dataset.zip

# Download the second dataset
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/cx268tmrwn-3.zip -O /kaggle/working/dataset.zip
!unzip /kaggle/working/dataset.zip -d /kaggle/working/
!rm /kaggle/working/dataset.zip


'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [6]:
import pandas as pd
import os

isolated_words_dictionary_path = "/kaggle/working/YembaTones An Annotated Dataset for Tonal and Syllabic Analysis of the Yemba Language/Yemba_Dataset/metadata/isolated_words_dictionary.xlsx"
word_corpus_path = "/kaggle/working/YembaEGRA/YembaEGRA/YembaEGRA/metadata/words_corpus .csv"

isolated_words_df = pd.read_excel(isolated_words_dictionary_path)
word_corpus_df = pd.read_csv(word_corpus_path)

df = pd.DataFrame(columns=["Yemba", "path"])
new_rows = []

yemba_tones_base = "/kaggle/working/YembaTones An Annotated Dataset for Tonal and Syllabic Analysis of the Yemba Language/Yemba_Dataset/audios"
yemba_egra_tones_base = "/kaggle/working/YembaEGRA/YembaEGRA/YembaEGRA/audio"

speaker_length = 11
groups_length = 149

# Build rows from YembaTones
for i in range(1, speaker_length + 1):
    s_path = os.path.join(yemba_tones_base, f"speaker_{i}")
    for j in range(1, groups_length + 1):
        wav_count = 0
        g_path = os.path.join(s_path, f"group_{j}")
        if not os.path.exists(g_path):
            continue
        # Filter once outside the loop
        filtered_df = isolated_words_df.loc[isolated_words_df['GroupeId'] == j]
        max_len = len(filtered_df)
        
        for root, dirs, files in os.walk(g_path):
            for file in files:
                if file.endswith(".wav"):
                    if wav_count < max_len:
                        row_index = filtered_df.index[wav_count]
                        save_path = os.path.join(root, file)
                        new_rows.append({
                            "Yemba": filtered_df["Yemba"].iloc[wav_count],
                            "path": save_path
                        })
                        wav_count += 1

# Build rows from YembaEGRA
ci_length = 8
word_length = 60

for i in range(1, ci_length + 1):
    s_path = os.path.join(yemba_egra_tones_base, f"CI{i}")
    for j in range(1, word_length + 1):
        g_path  = os.path.join(s_path, f"W{j}")
        if not os.path.exists(g_path):
            continue
        for root, dirs, files in os.walk(g_path):
            for file in files:
                if file.endswith(".wav"):
                    save_path = os.path.join(root, file)
                    filtered = word_corpus_df.loc[word_corpus_df['id_word'] == j]
                    if len(filtered) > 0:
                        yemba_value = filtered["Yemba"].iloc[0]
                        new_rows.append({
                            "Yemba": yemba_value,
                            "path": save_path
                        })

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# Shuffle and split
df = df.sample(frac=1).reset_index(drop=True)




FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/YembaTones An Annotated Dataset for Tonal and Syllabic Analysis of the Yemba Language/Yemba_Dataset/metadata/isolated_words_dictionary.xlsx'

In [7]:
import soundfile as sf
from datasets import Dataset, DatasetDict, Audio

# 1) Read your CSV files
train_df = pd.read_csv("training_data.csv")
test_df = pd.read_csv("testing_data.csv")

# 2) Ensure required columns exist
required_cols = {"path", "Yemba"}
if not required_cols.issubset(train_df.columns):
    raise ValueError("Train CSV must contain 'path' and 'Yemba' columns.")
if not required_cols.issubset(test_df.columns):
    raise ValueError("Test CSV must contain 'path' and 'Yemba' columns.")

# 3) Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 4) Verify audio paths and store them in the 'audio' column
def verify_and_process_audio_column(batch):
    audio_path = batch["path"]

    if not os.path.isfile(audio_path):
        print(f"File not found: {audio_path}. Skipping.")
        batch["audio"] = None
        return batch

    try:
        # Just to verify readability
        _data, _sr = sf.read(audio_path)
    except Exception as e:
        print(f"Invalid audio file: {audio_path} ({e}). Skipping.")
        batch["audio"] = None
        return batch

    batch["audio"] = {"path": audio_path}
    return batch

train_dataset = train_dataset.map(verify_and_process_audio_column)
test_dataset = test_dataset.map(verify_and_process_audio_column)

def filter_invalid(batch):
    return batch["audio"] is not None

train_dataset = train_dataset.filter(filter_invalid)
test_dataset = test_dataset.filter(filter_invalid)

# 5) Cast the valid 'audio' column to the Audio feature
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

# Remove original 'path' if you no longer need it:
train_dataset = train_dataset.remove_columns(["path"])
test_dataset = test_dataset.remove_columns(["path"])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(dataset)


c:\Users\USER\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'training_data.csv'

In [8]:
df.to_csv("new_data_path_file.csv")

NameError: name 'df' is not defined

In [8]:
df =df.sample(frac=1).reset_index(drop=True)
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df,test_size=0.1,shuffle=True)

train_data.to_csv("training_data.csv",index=False)
test_data.to_csv("testing_data.csv",index=False)

len(train_data),len(test_data)

(10282, 1143)

In [9]:
train_data, test_data = \
    pd.read_csv("training_data.csv"), pd.read_csv("testing_data.csv") \
    if os.path.isfile("training_data.csv") and os.path.isfile("testing_data.csv") \
    else (None, None)

if train_data is None or test_data is None:
    # If they don't exist, do train_test_split
    from sklearn.model_selection import train_test_split
    train_data, test_data = train_test_split(df, test_size=0.1, shuffle=True)
    train_data.to_csv("training_data.csv", index=False)
    test_data.to_csv("testing_data.csv", index=False)

print("Train size:", len(train_data), " Test size:", len(test_data))

Train size: 10282  Test size: 1143


In [12]:
import os
import pandas as pd
import soundfile as sf
from datasets import Dataset, DatasetDict, Audio

# 1) Read your CSV files
train_df = pd.read_csv("training_data.csv")
test_df = pd.read_csv("testing_data.csv")

# 2) Ensure required columns exist
required_cols = {"path", "Yemba"}
if not required_cols.issubset(train_df.columns):
    raise ValueError("Train CSV must contain 'path' and 'Yemba' columns.")
if not required_cols.issubset(test_df.columns):
    raise ValueError("Test CSV must contain 'path' and 'Yemba' columns.")

# 3) Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 4) Verify audio paths and store them in the 'audio' column
def verify_and_process_audio_column(batch):
    audio_path = batch["path"]

    # Check if file exists
    if not os.path.isfile(audio_path):
        # Mark audio as None if missing
        print(f"File not found: {audio_path}. Skipping.")
        batch["audio"] = None
        return batch

    # Attempt reading with soundfile just to confirm it's valid
    try:
        # We won't keep data in memory here; just verifying readability
        _data, _sr = sf.read(audio_path)
    except Exception as e:
        # Mark audio as None if it's corrupted or unsupported
        print(f"Invalid audio file: {audio_path} ({e}). Skipping.")
        batch["audio"] = None
        return batch

    # If valid, store path in 'audio'
    batch["audio"] = {"path": audio_path}
    return batch

train_dataset = train_dataset.map(verify_and_process_audio_column)
test_dataset = test_dataset.map(verify_and_process_audio_column)

# 5) Optionally filter out rows with invalid audio
def filter_invalid(batch):
    return batch["audio"] is not None

train_dataset = train_dataset.filter(filter_invalid)
test_dataset = test_dataset.filter(filter_invalid)

# 6) Cast the valid 'audio' column to the Audio feature
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

# 7) Remove the original 'path' column
train_dataset = train_dataset.remove_columns(["path"])
test_dataset = test_dataset.remove_columns(["path"])

# 8) Combine into a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(dataset)

Map:   0%|          | 0/10282 [00:00<?, ? examples/s]

Invalid audio file: /kaggle/working/YembaTones An Annotated Dataset for Tonal and Syllabic Analysis of the Yemba Language/Yemba_Dataset/audios/speaker_9/group_81/spkr_9_group_81_statement_1.wav (Error opening '/kaggle/working/YembaTones An Annotated Dataset for Tonal and Syllabic Analysis of the Yemba Language/Yemba_Dataset/audios/speaker_9/group_81/spkr_9_group_81_statement_1.wav': Format not recognised.). Skipping.


Map:   0%|          | 0/1143 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10282 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1143 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Yemba', 'audio'],
        num_rows: 10281
    })
    test: Dataset({
        features: ['Yemba', 'audio'],
        num_rows: 1143
    })
})


In [13]:
!pip install bnunicodenormalizer
from bnunicodenormalizer import Normalizer

bnorm = Normalizer()

def normalize_text(batch):
    words = batch["Yemba"].split()
    norm_words = []
    for w in words:
        norm_dict = bnorm(w)  # returns {"original": ..., "normalized": ...}
        if norm_dict and "normalized" in norm_dict:
            norm_words.append(norm_dict["normalized"])
    batch["Yemba"] = " ".join(norm_words)
    return batch

dataset = dataset.map(normalize_text)


Map:   0%|          | 0/10281 [00:00<?, ? examples/s]

TypeError: sequence item 0: expected str instance, NoneType found

In [ ]:
import json

# We'll reuse the CSV we built or dataset to collect unique characters
# However, here you use the 'new_data_path_file.csv' from your code:
csv_path = "new_data_path_file.csv"

data_char = pd.read_csv(csv_path)

if 'Yemba' not in data_char.columns:
    raise ValueError("Ensure the CSV file has a 'Yemba' column for building vocab.")

words = data_char['Yemba'].dropna().unique()

char_vocab = set("".join(words))
char_vocab = sorted(list(char_vocab))

special_tokens = ["[UNK]", "[PAD]"]
char_vocab = special_tokens + char_vocab

char_to_index = {char: idx for idx, char in enumerate(char_vocab)}

vocab_file = "yemba_vocab.json"
with open(vocab_file, "w") as f:
    json.dump(char_to_index, f, ensure_ascii=False, indent=4)

print("Vocabulary size:", len(char_vocab))
print("Example chars:", char_vocab)


Vocabulary size: 47
Example chars: ['[UNK]', '[PAD]', ' ', "'", '-', 'A', 'E', 'K', 'L', 'M', 'N', 'S', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ŋ', 'ŋ', 'ǝ', 'ɔ', 'ɛ', 'ʉ', 'ʼ', '̀', '́', '̄', '’']


In [14]:
# Clone the repository
!git lfs install
!git clone https://huggingface.co/facebook/wav2vec2-large-xlsr-53 /kaggle/working/wav2vec2-large-xlsr-53

Git LFS initialized.
Cloning into '/kaggle/working/wav2vec2-large-xlsr-53'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36 (from 1)
Unpacking objects: 100% (36/36), 5.35 KiB | 608.00 KiB/s, done.
Filtering content: 100% (2/2), 2.36 GiB | 143.46 MiB/s, done.


In [27]:
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2ForCTC,
Wav2Vec2Config
)

with open("yemba_vocab.json", "r") as f:
    yemba_vocab = json.load(f)

tokenizer = Wav2Vec2CTCTokenizer(
    vocab_file="yemba_vocab.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token="|"  # or another if you prefer
)

# Load pretrained feature extractor
model_name = "/kaggle/working/wav2vec2-large-xlsr-53"

# 1) Load the base config
config = Wav2Vec2Config.from_pretrained(model_name)
# 2) Override the vocab_size
config.vocab_size = tokenizer.vocab_size  # e.g., 47 or whatever yours is

# 3) Create model with new config
model = Wav2Vec2ForCTC.from_pretrained(
    model_name,
    config=config,                # use updated config
    ignore_mismatched_sizes=True  # allows final linear layer to resize
)
processor.save_pretrained("./custom-ssa-processor")

# Load the Wav2Vec2 model
model = Wav2Vec2ForCTC.from_pretrained(model_name)
print("Custom processor and model successfully loaded!")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /kaggle/working/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /kaggle/working/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Custom processor and model successfully loaded!


In [28]:
import numpy as np

def prepare_dataset(batch):
    """
    Prepares audio and text for Wav2Vec2 training.
    dataset must have 'audio' (dict with 'array', 'sampling_rate') and 'Yemba' (string).
    """
    audio_array = batch["audio"]["array"]
    sampling_rate = batch["audio"]["sampling_rate"]

    # Process input audio
    inputs = processor(audio_array, sampling_rate=sampling_rate)
    batch["input_values"] = inputs.input_values[0]
    batch["input_length"] = len(batch["input_values"])

    # Process labels
    labels = processor(text=batch["Yemba"]).input_ids
    batch["labels"] = labels

    return batch


In [29]:
def trim_silence(batch):
    """
    Example function that tries to remove leading/trailing zeros.
    Use caution: make sure it doesn't create empty arrays.
    """
    arr = batch["input_values"]
    if len(arr) == 0:
        return batch

    # naive threshold-based trimming
    threshold = 30
    _max = max(abs(x) for x in arr) if len(arr) > 0 else 0
    left_idx = 0
    right_idx = len(arr) - 1

    # find left cut
    while left_idx < len(arr):
        if abs(arr[left_idx]) * threshold > _max:
            break
        left_idx += 1

    # find right cut
    while right_idx >= 0:
        if abs(arr[right_idx]) * threshold > _max:
            break
        right_idx -= 1

    trimmed = arr[left_idx:right_idx+1]
    batch["input_values"] = trimmed
    batch["input_length"] = len(trimmed)
    return batch

# 1) Map prepare_dataset to create input_values & labels
dataset_train = dataset["train"].map(prepare_dataset, remove_columns=dataset["train"].column_names)
dataset_validation = dataset["test"].map(prepare_dataset, remove_columns=dataset["test"].column_names)

# 2) Filter out empty labels
def filter_empty_labels(batch):
    return len(batch["labels"]) > 0

dataset_train = dataset_train.filter(filter_empty_labels)
dataset_validation = dataset_validation.filter(filter_empty_labels)

# 3) (Optional) Trim silence
dataset_train = dataset_train.map(trim_silence)
dataset_validation = dataset_validation.map(trim_silence)

# 4) Filter out any zero-length audio after trimming
def filter_zero_audio(batch):
    return batch["input_length"] > 0

dataset_train = dataset_train.filter(filter_zero_audio)
dataset_validation = dataset_validation.filter(filter_zero_audio)

print(dataset_train, dataset_validation)


Map:   0%|          | 0/10281 [00:00<?, ? examples/s]

Map:   0%|          | 0/1143 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10281 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1143 [00:00<?, ? examples/s]

Map:   0%|          | 0/10281 [00:00<?, ? examples/s]

Map:   0%|          | 0/1143 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10281 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1143 [00:00<?, ? examples/s]

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 10281
}) Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 1143
})


In [30]:
import torch
from dataclasses import dataclass
from typing import List, Dict, Union
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": f["input_values"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [31]:
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Replace -100 in labels with pad_token_id so we can decode
    labels = pred.label_ids
    labels[labels == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(labels, skip_special_tokens=True)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Optionally freeze the feature encoder
model.freeze_feature_encoder()
print("Feature encoder frozen.")


Feature encoder frozen.


In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [24]:
from huggingface_hub import create_repo

repo_name = "my-yemba-model_1"  # choose your unique name
create_repo(repo_id=repo_name, private=False)  #

RepoUrl('https://huggingface.co/Ngoune/my-yemba-model_1', endpoint='https://huggingface.co', repo_type='model', repo_id='Ngoune/my-yemba-model_1')

In [34]:
model = Wav2Vec2ForCTC.from_pretrained(
    model_name,
    vocab_size=tokenizer.vocab_size,
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /kaggle/working/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model.config.vocab_size == tokenizer.vocab_size

True

In [ ]:
from transformers import TrainingArguments, Trainer
repo_name = "my-yemba-model_1"  # choose your unique name

my_token = ""  # put your real token if pushing to the hub

training_args = TrainingArguments(
    output_dir=repo_name,
    report_to="none",
    # If pushing to hub, set these:
    # hub_token=my_token,
    # push_to_hub=True,
    # hub_model_id=repo_name,

    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    gradient_checkpointing=True,
    save_steps=9630,
    eval_steps=1500,
    logging_strategy="epoch",
    learning_rate=5e-7,
    weight_decay=0.0000025,
    warmup_steps=500,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    tokenizer=processor.feature_extractor,  # or just processor if it supports it
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-40-5d23fc0048ce>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in 

Step,Training Loss,Validation Loss,Wer
1500,2345.724000,433.756439,1.000000
3000,1552.068300,316.668823,1.000000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
trainer.push_to_hub()